In [14]:
from tensorflow.keras.models import load_model

model = load_model('D:\proand\predict\model.h5')

In [2]:
import pandas as pd
df = pd.read_csv('D:\proand\data\ingredient.csv')

In [3]:
import random

def random_word_deletion(sentence, deletion_prob=0.1):
    '''
    Add Random Word Deletion for Data Augmentation
    A sentence parameter is the values in ingredients column of the dataset
    '''
    if isinstance(sentence, str):
        words = sentence.split()
        new_words = [word for word in words if random.random() > deletion_prob]
        if not new_words:
            return sentence
        else:
            return ' '.join(new_words)
    else:
        return sentence

def shuffle_ingredients(sentence):
    '''
    Shuffle the ingredients' orders for Data Augmentation
    A sentence parameter is the values in ingredients column of the dataset
    '''
    if isinstance(sentence, str):
        words = sentence.split()
        random.shuffle(words)
        return ' '.join(words)
    else:
        return sentence


In [4]:
df['ingredients'] = df['ingredients'].apply(random_word_deletion)
df['ingredients'] = df['ingredients'].apply(shuffle_ingredients)

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['ingredients'])
encoded_ingredients = tokenizer.texts_to_sequences(df['ingredients'])
max_length = max([len(seq) for seq in encoded_ingredients])
padded_ingredients = pad_sequences(encoded_ingredients, maxlen=max_length, padding='post')


In [7]:
label_encoder = LabelEncoder()
ratings_encoded = label_encoder.fit_transform(df['rating'])


In [8]:
X_train, X_test, y_train, y_test = train_test_split(padded_ingredients, ratings_encoded, test_size=0.2, random_state=42)

In [15]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(
    x=[X_train, y_train, X_train],
    y=y_train,
    epochs=300,
    batch_size=16,
    validation_split=0.15,
    callbacks=[early_stopping]
)

Epoch 1/300
7/7 [==============================] - 3s 137ms/step - loss: 1.6749 - accuracy: 0.3423 - val_loss: 1.5578 - val_accuracy: 0.4000
Epoch 2/300
7/7 [==============================] - 0s 22ms/step - loss: 1.6607 - accuracy: 0.3333 - val_loss: 1.5578 - val_accuracy: 0.4000
Epoch 3/300
7/7 [==============================] - 0s 22ms/step - loss: 1.6408 - accuracy: 0.3514 - val_loss: 1.5398 - val_accuracy: 0.5000
Epoch 4/300
7/7 [==============================] - 0s 22ms/step - loss: 1.6337 - accuracy: 0.3333 - val_loss: 1.5357 - val_accuracy: 0.4000
Epoch 5/300
7/7 [==============================] - 0s 21ms/step - loss: 1.6212 - accuracy: 0.3514 - val_loss: 1.5218 - val_accuracy: 0.4000
Epoch 6/300
7/7 [==============================] - 0s 21ms/step - loss: 1.6385 - accuracy: 0.3423 - val_loss: 1.5328 - val_accuracy: 0.4500
Epoch 7/300
7/7 [==============================] - 0s 21ms/step - loss: 1.6132 - accuracy: 0.3514 - val_loss: 1.5692 - val_accuracy: 0.5000
Epoch 8/300
7/7 [==

In [16]:
# model.save('D:\proand\predict\model.h5')

d:\proand\env\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
